In [1]:
import math

import os
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from collections import defaultdict
from time import sleep

import numpy as np
import multiprocessing, random
from scipy.integrate import simps, dblquad
import pickle
from gtda.images import *
from gtda.homology import CubicalPersistence
import persistence_diagram, utils
import tensorflow as tf


In [2]:
train_images, train_labels, test_images, test_labels = utils.get_mnist_data()
train_images = train_images[0:10]

## Set the params of the filtrations to extract

# Height filtration
num_of_vects = 6
angles = np.linspace(0, math.pi/2, num_of_vects)
dirs = [[round(math.cos(theta),2),round(math.sin(theta),2)] for theta in angles]
dirs = np.array(dirs)

# Radial filtration
center = np.array([[10,10], [10,20], [20,10], [20,20]])
radius = np.array([5, 10, 15])

# Erosion filtration
n_iter_er = np.array([1,2,3,50])

# Dilation filtration
n_iter_dil = np.array([1,3,5,10,50])

params = {'cubical' : None,
         'height': dirs,
         'radial': {'center' : center,
                    'radius' : radius
                    },
         'erosion': n_iter_er,
         'dilation': n_iter_dil
         }

In [3]:
PD = persistence_diagram.PDiagram(train_images, params, images_id= 'mnist_train', man_dim=9)
print(PD.save_dir)
f=open(PD.save_dir+ 'dgms.pkl','rb')
[fil_params, dgms_dict] = pickle.load(f)

data/diagrams/mnist_train/


In [4]:
pds = PD.get_pds()
#vpds = PD.get_vectorized_pds()

In [6]:
train_data = dict()
for ind in pds.keys():
    im_data = dict()
    cnt = 0
    for filtration in pds[ind].keys():
        for p in pds[ind][filtration].keys():
            im_data[cnt] = pds[ind][filtration][p]
            cnt +=1
    train_data[ind] = im_data

In [9]:
man_dim=2
def pad_diagrams(dgms):
    '''
        Adds zeros points to the diagrams to make the of equal size; tensorflow cant handle inputs with varied size
    '''
    # Get the highest number of points in a PD
    max_num_of_pnts = 0
    for ind in dgms.keys():
        for filtration in dgms[ind].keys():
            for par_ind in dgms[ind][filtration].keys():
                dgm = dgms[ind][filtration][par_ind]
                max_num_of_pnts = max(max_num_of_pnts, dgm.shape[0])

    # Pad
    x = []
    for ind in dgms.keys():
        for filtration in dgms[ind].keys():
            for par_ind in dgms[ind][filtration].keys():
                dgm = dgms[ind][filtration][par_ind]
                x_padded = np.zeros((max_num_of_pnts, man_dim+1)) # plus one cus the first item of each poin is the homology class
                x_padded[:dgm.shape[0],:] = dgm
                x.append(x_padded)
    return np.array(x)

pad = pad_diagrams(pds)

MemoryError: Unable to allocate 2.89 GiB for an array with shape (1680000, 77, 3) and data type float64

In [ ]:
PD = persistence_diagram.PDiagram(test_images, params, images_id= 'mnist_test', man_dim=9)
pds = PD.get_pds()
#vpds = PD.get_vectorized_pds()

In [4]:
enumerate([1,5,10])